# PhotoLoader ICEDID
> Taking a closer look at this ICEDID loader

- toc: true 
- badges: true
- categories: [icedid,bokbot,photoloader,config]


## Overview

Photoloader is the initial loader stage used to load ICEDID, 


ICEDID was originally used for banking credential theft with a later pivot as a reconnaissance tool for pre-ransomware intrusions. The webinjects used for credential theft are still active though this malware is most often associated with ransomware incidents. 

According to [Proofpoint](https://www.proofpoint.com/us/blog/threat-insight/fork-ice-new-era-icedid) there is a fork of ICEDID that does not have webinject capability and is possibly developed by three separate actors...

> **Standard IcedID Variant** – The variant most commonly observed in the threat landscape and used by a variety of threat actors.

> **Lite IcedID Variant** – New variant observed as a follow-on payload in November Emotet infections that does not exfiltrate host data in the loader checkin and a bot with minimal functionality.

> **Forked IcedID Variant** – New variant observed by Proofpoint researchers in February 2023 used by a small number of threat actors which also delivers the bot with minimal functionality.

### References

- [DFIRReport: ICEDID -> Quantum ransomware](https://thedfirreport.com/2023/04/03/malicious-iso-file-leads-to-domain-wide-ransomware/)
- [ICEDIDs network infrastructure is alive and well](https://www.elastic.co/security-labs/icedids-network-infrastructure-is-alive-and-well)
- [ICEDID Configuration Extractor](https://www.elastic.co/security-labs/icedid-configuration-extractor)
- [Fork in the Ice: The New Era of IcedID](https://www.proofpoint.com/us/blog/threat-insight/fork-ice-new-era-icedid)
- [icedid_peloader.py](https://github.com/c3rb3ru5d3d53c/mwcfg-modules/blob/f1064aea63d11b5069a1839cf2b9d10d43cee1aa/icedid/peloader/icedid_peloader.py)
- [New version of IcedID Trojan uses steganographic payloads](https://www.malwarebytes.com/blog/news/2019/12/new-version-of-icedid-trojan-uses-steganographic-payloads)

### Samples

- new loader sample (dfir report)`2db4fadfb2565fd9474e4d5303f953e96ac248de3267014c32e8a669e7e600e0` [UnpacMe](https://www.unpac.me/results/ae0e555a-2ef1-4e42-8fa2-f7472e82d7ef)
- older sample `963397cec08790b25ff273cbe4b133634ae045d5ff8a4492e6f585f2ad14db65`[UnpacMe](https://www.unpac.me/results/635b6739-0c39-402d-b456-c3cc39412395)
- old unpacked 32bit photoloader `1b01700425c30c2c498718966aee96cfdebacc2f6167576f7aa56e3f43ec3282` [malpedia](https://malshare.com/sample.php?action=detail&hash=1b01700425c30c2c498718966aee96cfdebacc2f6167576f7aa56e3f43ec3282)

## Analysis

It looks like the Malpedia `photoloader` yara rules are a bit too loose and match the newer "gzip" variant of the loader. The config location/encryption is different between these two loaders and `photoloader` has not been used in a few years. We are going to create a new rule that will be used to only match the newer variants.


### Rule

This rule is **heavily** influenced by the elastic rules in their [config extractor](https://www.elastic.co/security-labs/icedid-configuration-extractor)

```c
rule icedid_loader {        
    strings:
        $a1 = "; _gat=" wide fullword
        $a2 = "; _ga=" wide fullword
        $a3 = "; _u=" wide fullword
        $a4 = "; __io=" wide fullword
        $a5 = "; _gid=" wide fullword
        $a6 = "loader_dll_64.dll" ascii fullword
        $config_decryption1 = {45 33 C0 4C 8D 0D ?? ?? ?? ?? 49 2B C9 4B 8D 14 08 49 FF C0 8A 42 ?? 32 02 88 44 11 ?? 49 83 F8 }
		$config_decryption2 = { 00 42 8A 44 01 ?? 42 32 04 01 88 44 0D ?? 48 FF C1 48 83 F9 }
        condition:
            filesize < 60000 and
            (
                (3 of ($a*) and $config_decryption1) or
                $config_decryption2
            )
}

```

### Config Extractor

This is a modified version of the [elastic config extractor](https://www.elastic.co/security-labs/icedid-configuration-extractor)



In [18]:
import pefile
import re
import struct

file_data = open('/tmp/samples/963397cec08790b25ff273cbe4b133634ae045d5ff8a4492e6f585f2ad14db65', 'rb').read()
pe = pefile.PE(data = file_data)

In [23]:
IMAGE_SCN_CNT_CODE = 0x00000020

def xor(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)])
    return bytes(out)

def is_ascii(s):
    return all((c < 128 and c > 39) or c == 0 for c in s)


key = None
domain = None
campaign_id = None


mapped = False
if pe.sections[0].get_data()[:100] == b'\x00'*100:
    print("Mapped!")
    mapped = True

for s in pe.sections:
    if (s.Characteristics & IMAGE_SCN_CNT_CODE) == 0:
        if mapped:
            section_data = file_data[s.VirtualAddress:s.VirtualAddress +256]
        else:
            section_data = s.get_data()
        if len(section_data) < 250:
            print("Section too small")
            continue
        # This is a hack to skip stuff that doesn't look like a key
        tmp_key = section_data[:32]
        if b'\x00'*10 in tmp_key:
            print("Too many nulls in key")
            continue
        data = section_data[64:96]
        tmp_config = xor(data, tmp_key)
        domain = None
        try:
            domains = tmp_config[4:]
            domains = domains.split(b"\x00")
            if not is_ascii(domains[0]):
                        continue
            domain = domains[0].decode("UTF-8")
        except:
            print("Domain decode error")
            continue
        if len(domain) < 5:
            print("Domain too small")
            continue
        # If we are here we have a config! 
        campaign_id = struct.unpack('<I', tmp_config[:4])[0]
        key = tmp_key.hex()
        break
        
assert key is not None
assert domain  is not None
assert campaign_id is not None
        
config = {
            "campaign_id": campaign_id,
            "domains": domain,
            "key": key,
         }   
        
        
print(config)

Mapped!
Too many nulls in key
{'campaign_id': 3581911946, 'domains': 'smockalifatori.com', 'key': '5e845c90daccb7f15a824ddf17ebccb65094b386fa909bf6c802f42a295e5dc1'}


In [28]:
def is_ascii(s):
    return all((c < 128 and c > 39) or c == 0 for c in s)

def extract_config(file_path):
    file_data = open(file_path, 'rb').read()
    pe = pefile.PE(data = file_data)
    
    mapped = False
    if pe.sections[0].get_data()[:100] == b'\x00'*100:
        #print("Mapped!")
        mapped = True
    
    key = None
    domain = None
    campaign_id = None

    try:
        for s in pe.sections:
            if (s.Characteristics & IMAGE_SCN_CNT_CODE) == 0:
                if mapped:
                    section_data = file_data[s.VirtualAddress:s.VirtualAddress +256]
                else:
                    section_data = s.get_data()
                if len(section_data) < 250:
                    #print("Section too small")
                    continue
                # This is a hack to skip stuff that doesn't look like a key
                tmp_key = section_data[:32]
                if b'\x00'*10 in tmp_key:
                    #print("Too many nulls in key")
                    continue
                data = section_data[64:96]
                tmp_config = xor(data, tmp_key)
                domain = None
                try:
                    domains = tmp_config[4:]
                    domains = domains.split(b"\x00")
                    if not is_ascii(domains[0]):
                        continue
                    domain = domains[0].decode("UTF-8")
                except:
                    #print("Domain decode error")
                    continue
                if len(domain) < 5:
                    #print("Domain too small")
                    continue
                # If we are here we have a config! 
                campaign_id = struct.unpack('<I', tmp_config[:4])[0]
                key = tmp_key.hex()
                break

        assert key is not None
        assert domain  is not None
        assert campaign_id is not None

        config = {
                    "campaign_id": campaign_id,
                    "domains": domain,
                    "key": key,
                 }   
    except:
        return {}
    return config




# import required module
import os
# assign directory
directory = '/tmp/samples/'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        config = extract_config(f)
        print(config)

/tmp/samples/884cdf248d0235d77adc1d88603d460d64c88c517d5e571b75749be42364d6a8
{'campaign_id': 3248465841, 'domains': 'qsertopinajil.com', 'key': 'e999037e2b4084f0c1284ac991eca172030d99a0fd13ad6061af36c0d26bd1c0'}
/tmp/samples/a2158fb6574d9d8f473eee19b6cba91f2d5c0fc5289e9245bb4d290380e22226
{'campaign_id': 2615141838, 'domains': 'olifamagaznov.com', 'key': '287130e4e0b4080bf367a2a3aaede31b5d652977f5e8a702b1c25b1afc7c2368'}
/tmp/samples/056de2c7a57fb4022d19398c6fc2676565afcda5e1f05ba0d91e58284e36d682
{'campaign_id': 133894510, 'domains': 'restorahlith.com', 'key': 'aa677c588cb603932a4965e66ca04fbfb85784e26eb18cb73555537c554a0568'}
/tmp/samples/f1a6325da85adcae0b21cd02592dfc4747fbe5b4eb428dd515000e35cc4e7f47
{'campaign_id': 3278418257, 'domains': 'ariopolanetyoa.com', 'key': '7bbf03f22f1a464224f55dd9c01de97d218f26e8058e48b749ba1892293b99b1'}
/tmp/samples/4cda20be09dd99ab0ccf618a6c1c62122f0c484fb5925031b6ab3e7ce2f016ae
{}
/tmp/samples/98984bc4bd9af65911d9102bde5cae341ffb9bcc913aca1fe690934